In [2]:
from django.contrib.gis.db.models.functions import Area
from django.contrib.gis.geos import Polygon
from django.db.models import Case, CharField, DecimalField, F, Q, Sum, Value, When
from django.db.models.functions import Cast, Concat

# from django.db.models import  F, Max, Min, Q, ,   
# from django.db.models.functions import Coalesce,  Cast

from project.models import Project
from public_data.models import OcsgeDiff, Ocsge, ZoneUrba

from prettytable import PrettyTable


zone_urba = ZoneUrba.objects.get(pk=599878)
diagnostic = Project.objects.get(pk=1504)
last_year_ocsge = diagnostic.last_year_ocsge
zone_urba, diagnostic

(<ZoneUrba: ZoneUrba object (599878)>,
 <Project: Diagnostic de CA Grand Auch Coeur de Gascogne>)

In [2]:
Zero = Area(Polygon(((0, 0), (0, 0), (0, 0), (0, 0)), srid=2154))
Zero

Area(Value(<Polygon object at 0x7f2fb52eac90>))

In [3]:
(
    OcsgeDiff.objects.intersect(zone_urba.mpoly)
    .filter(year_old__gte=diagnostic.first_year_ocsge, year_new__lte=diagnostic.last_year_ocsge)
    .filter(Q(is_new_artif=True) | Q(is_new_natural=True))
    .annotate(
        period=Cast(Concat("year_old", Value(" - "), "year_new"), CharField(max_length=15)),
        area_artif=Case(When(is_new_artif=True, then=F("intersection_area")), default=Zero),
        area_renat=Case(When(is_new_natural=True, then=F("intersection_area")), default=Zero),
    )
    .values("period")
    .annotate(
        new_artif=Cast(Sum("area_artif") / 10000, DecimalField(max_digits=15, decimal_places=2)),
        new_nat=Cast(Sum("area_renat") / 10000, DecimalField(max_digits=15, decimal_places=2)),
    )
)


<QuerySet [{'period': '2016 - 2019', 'new_artif': Decimal('12.57'), 'new_nat': Decimal('0.00')}]>

In [29]:
(
    Ocsge.objects.intersect(diagnostic.combined_emprise)
    .filter(is_artificial=True, year=last_year_ocsge)
    .aggregate(area=Sum("intersection_area") / 10000)
)

{'area': 5000.653403862377}

In [16]:
qs = (
    Ocsge.objects.intersect(zone_urba.mpoly)
    .filter(is_artificial=True, year=last_year_ocsge)
    .annotate(
        code_prefix=F(f"matrix__couverture__code_prefix"),
        label=F(f"matrix__couverture__label"),
        label_short=F(f"matrix__couverture__label_short"),
        map_color=F(f"matrix__couverture__map_color"),
    )
    .order_by("code_prefix", "label", "label_short", "map_color")
    .values("code_prefix", "label", "label_short", "map_color")
    .annotate(surface=Sum("intersection_area") / 10000)
)
sum([_['surface'] for _ in qs])

12.776170209555929

In [17]:
qs = (
    Ocsge.objects.intersect(zone_urba.mpoly)
    .filter(is_artificial=True, year=diagnostic.last_year_ocsge)
    .annotate(
        code_prefix=F("matrix__usage__code_prefix"),
        label=F("matrix__usage__label"),
        label_short=F("matrix__usage__label_short"),
        map_color=F("matrix__usage__map_color"),
    )
    .order_by("code_prefix", "label", "label_short", "map_color")
    .values("code_prefix", "label", "label_short", "map_color")
    .annotate(surface=Sum("intersection_area") / 10000)
)
sum([_['surface'] for _ in qs])

12.776170209555929

27

In [25]:
sol = "usage"
qs = (
    OcsgeDiff.objects.intersect(diagnostic.combined_emprise)
    .filter(
        year_old__gte=diagnostic.analyse_start_date,
        year_new__lte=diagnostic.analyse_end_date,
    )
    .filter(Q(is_new_artif=True) | Q(is_new_natural=True))
    .annotate(
        code_prefix=Case(
            When(is_new_artif=True, then=F(f"new_matrix__{sol}__code_prefix")),
            default=F(f"old_matrix__{sol}__code_prefix"),
        ),
        label=Case(
            When(is_new_artif=True, then=F(f"new_matrix__{sol}__label")),
            default=F(f"old_matrix__{sol}__label"),
        ),
        label_short=Case(
            When(is_new_artif=True, then=F(f"new_matrix__{sol}__label_short")),
            default=F(f"old_matrix__{sol}__label_short"),
        ),
        area_artif=Case(When(is_new_artif=True, then=F("intersection_area")), default=Zero),
        area_renat=Case(When(is_new_natural=True, then=F("intersection_area")), default=Zero),
    )
    .order_by("code_prefix", "label", "label_short")
    .values("code_prefix", "label", "label_short")
    .annotate(
        artif=Cast(Sum("area_artif") / 10000, DecimalField(max_digits=15, decimal_places=2)),
        renat=Cast(Sum("area_renat") / 10000, DecimalField(max_digits=15, decimal_places=2)),
    )
)
qs.count(), qs

(416,
 <QuerySet [{'code_prefix': 'US1.1', 'label': 'Agriculture', 'label_short': 'Agriculture', 'artif': Decimal('3.66'), 'renat': Decimal('0.34')}, {'code_prefix': 'US2', 'label': 'Secondaire', 'label_short': 'Secondaire', 'artif': Decimal('7.82'), 'renat': Decimal('0.00')}, {'code_prefix': 'US235', 'label': 'Production secondaire; tertiaire et usage résidentiel', 'label_short': 'Production secondaire; tertiai...', 'artif': Decimal('0.90'), 'renat': Decimal('0.30')}, {'code_prefix': 'US3', 'label': 'Tertiaire', 'label_short': 'Tertiaire', 'artif': Decimal('18.95'), 'renat': Decimal('0.05')}, {'code_prefix': 'US4.1.1', 'label': 'Routier', 'label_short': 'Routier', 'artif': Decimal('1.91'), 'renat': Decimal('0.08')}, {'code_prefix': 'US4.1.2', 'label': 'Ferré', 'label_short': 'Ferré', 'artif': Decimal('0.62'), 'renat': Decimal('0.00')}, {'code_prefix': 'US4.3', 'label': 'Réseaux d’utilité publique', 'label_short': 'Réseaux d’utilité publique', 'artif': Decimal('0.75'), 'renat': Decimal

In [28]:
headers = [
    "code_prefix",
    "label",
    "label_short",
    "artif",
    "renat",
]
x = PrettyTable(field_names=headers)

x.add_rows([
    [_[t] for t in headers]
    for _ in qs
])

x

code_prefix,label,label_short,artif,renat
US1.1,Agriculture,Agriculture,3.66,0.34
US2,Secondaire,Secondaire,7.82,0.00
US235,Production secondaire; tertiaire et usage résidentiel,Production secondaire; tertiai...,0.90,0.30
US3,Tertiaire,Tertiaire,18.95,0.05
US4.1.1,Routier,Routier,1.91,0.08
US4.1.2,Ferré,Ferré,0.62,0.00
US4.3,Réseaux d’utilité publique,Réseaux d’utilité publique,0.75,0.62
US5,Résidentiel,Résidentiel,18.39,2.67
US6.1,Zones en transition,Zones en transition,15.59,2.20
US6.2,Zones abandonnées,Zones abandonnées,0.02,0.02


In [36]:
qs = (
    Ocsge.objects.intersect(diagnostic.combined_emprise)
    .filter(year__gte=diagnostic.analyse_start_date, year__lte=diagnostic.analyse_end_date)
    .order_by("year")
    .distinct()
    .values_list("year", flat=True)
)
qs

<QuerySet [2016, 2019]>

In [37]:
millesimes = qs
{"first": min(millesimes), "last": max(millesimes)}

{'first': 2016, 'last': 2019}

In [4]:
from django.urls import reverse

reverse("project:map-pane-artif-zone-urba", kwargs={"project_id": diagnostic.id, "pk": 0})

'/project/1504/carte/detail-zone-urbaine/0'